## Imports

In [3]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [4]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

May. 13th May. 20th


In [5]:
filename = glob.glob('redfin*.csv')
filename = filename[0]

In [6]:
df = pd.read_csv(filename)
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [7]:
new_row = {'SALE TYPE':'PAST SALE', 
           'SOLD DATE':'May-10-2024',
           'PROPERTY TYPE':'Condo/Co-op', 
           'ADDRESS':'2901 Collins Ave # 1205', 
           'CITY':'Miami Beach',
           'STATE OR PROVINCE':'FL', 
           'ZIP OR POSTAL CODE':'33140', 
           'PRICE':4995000,
           'BEDS':2, 
           'BATHS':2.5,
           'LOCATION':'The Edition', 
           'SQUARE FEET':1450, 
           'LOT SIZE':None,
           'YEAR BUILT':2014, 
           'DAYS ON MARKET':None,
           '$/SQUARE FEET':3445, 
           'HOA/MONTH':None, 
           'STATUS':'Sold', 
           'NEXT OPEN HOUSE START TIME':'NA',
           'NEXT OPEN HOUSE END TIME':'NA', 
           'URL':'https://www.redfin.com/FL/Miami-Beach/2901-Collins-Ave-33140/unit-1205/home/45457156', 
           'SOURCE':'Manual Entry', 
           'MLS#':'A11456329', 
           'FAVORITE':'NA',
           'INTERESTED':'NA', 
           'LATITUDE':25.805127277324562, 
           'LONGITUDE': -80.12411666847869}

In [8]:
# Convert the new row to a DataFrame
new_row_df = pd.DataFrame([new_row])

# Concatenate the new row to the original DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_17741/3499859834.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row_df], ignore_index=True)


In [10]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [12]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

174


In [13]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$964,717


In [14]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$641


In [15]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$167,860,777


## Previous Week's Values

In [17]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
165
Input Previous Week Condo Average Sales Price:
850114
Input Previous Week Condo Average PSF:
560
Input Previous Week Condo Sales total (ex: 198_000_000)
140000000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [18]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [19]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [20]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [21]:
pd.set_option('display.max_columns',None)

In [22]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
133,PAST SALE,May-15-2024,Condo/Co-op,50 S Pointe Dr #2201,Miami Beach,FL,33139.0,10000000.0,3.0,4.5,CONTINUUM ON SOUTH BEACH,2872.0,NaN,2008.0,NaN,3482.0,4668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11513210,N,Y,25.767875,-80.132756,1,orange


In [23]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [24]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [25]:
m.save('index.html')

## Data snagger

In [26]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [27]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
133,PAST SALE,May-15-2024,Condo/Co-op,50 S Pointe Dr #2201,Miami Beach,FL,33139.0,10000000.0,3.0,4.5,CONTINUUM ON SOUTH BEACH,2872.0,NaN,2008.0,NaN,3482.0,4668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11513210,N,Y,25.767875,-80.132756,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [28]:
df_top_ten['PRICE'].iloc[0]

10000000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [29]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_052024


# CREATE TEMPLATE 

In [30]:
muni_set = set(df_top_ten['CITY'])

In [31]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [32]:
df_top_ten.reset_index(inplace=True,drop=True)

In [33]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [34]:
top_sale

'CONTINUUM ON SOUTH BEACH at 50 S Pointe Dr #2201 in Miami Beach'

In [29]:
# previous_week_condo_sales_total = 130_000_000

In [35]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [111]:
# df_top_ten

## RESUME HERE

In [36]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: CONTINUUM ON SOUTH BEACH closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $3,700,000M to $10,000,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Sunny Isles Beach, Miami Beach, Miami, Bal Harbour

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 174 condo sales totaling $167,860,777 million from May. 13th to May. 20th. The previous week, brokers closed 165 condo sales totaling $140,000,000.

Last week’s units sold for an average of $964,717, higher than the $850,114 sales average from the previous week. The avera

In [37]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [39]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: CONTINUUM ON SOUTH BEACH closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $3,700,000M to $10,000,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Sunny Isles Beach, Miami Beach, Miami, Bal Harbour

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 174 condo sales totaling $167,860,777 million from May. 13th to May. 20th. The previous week, brokers closed 165 condo sales totaling $140,000,000.

Last week’s units sold for an average of $964,717, higher than the $850,114 sales average from the previous week. The avera

In [53]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/4701-N-Meridian-Ave-33140/unit-512/home/79136328


In [48]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Bal-Harbour/10295-Collins-Ave-33154/unit-403/home/43449070


In [49]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/17875-Collins-Ave-33160/unit-3706/home/43362748


In [54]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/5775-Collins-Ave-33140/unit-1702/home/178632955


In [52]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Bal-Harbour/10295-Collins-Ave-33154/unit-403/home/43449070


In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,May-15-2024,Condo/Co-op,50 S Pointe Dr #2201,Miami Beach,FL,33139.0,10000000.0,3.0,4.5,CONTINUUM ON SOUTH BEACH,2872.0,NaN,2008.0,NaN,3482.0,4668.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11513210,N,Y,25.767875,-80.132756,1,orange,CONTINUUM ON SOUTH BEACH at 50 S Pointe Dr #22...
1,PAST SALE,May-15-2024,Condo/Co-op,4701 N Meridian Ave #512,Miami Beach,FL,33140.0,8500000.0,4.0,4.5,4701 NORTH MERIDIAN CONDO,4204.0,NaN,2019.0,NaN,2022.0,9350.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11545111,N,Y,25.822813,-80.130733,2,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
2,PAST SALE,May-14-2024,Condo/Co-op,17901 Collins Ave #2403,Sunny Isles Beach,FL,33160.0,7600000.0,4.0,5.0,Estates at Acqualina,3287.0,NaN,2022.0,NaN,2312.0,6125.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11539419,N,Y,25.941881,-80.120417,3,blue,Estates at Acqualina at 17901 Collins Ave #240...
3,PAST SALE,May-17-2024,Condo/Co-op,2811 S Bayshore Dr Unit 15D,Miami,FL,33133.0,7200000.0,3.0,3.5,C.G.B. SUBDIVISION,2546.0,NaN,2020.0,NaN,2828.0,5565.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11525122,N,Y,25.728751,-80.238013,4,blue,C.G.B. SUBDIVISION at 2811 S Bayshore Dr Unit ...
4,PAST SALE,May-17-2024,Condo/Co-op,5775 Collins Ave #1702,Miami Beach,FL,33140.0,6000000.0,2.0,3.0,AMD PL OF 2ND OCEAN FRONT,1612.0,NaN,2021.0,NaN,3722.0,4431.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5775-Col...,MARMLS,A11587566,N,Y,25.839257,-80.120395,5,blue,AMD PL OF 2ND OCEAN FRONT at 5775 Collins Ave ...
5,PAST SALE,May-16-2024,Condo/Co-op,15701 Collins Ave #2601,Sunny Isles Beach,FL,33160.0,5000000.0,3.0,4.5,15701 COLLINS CONDO,3080.0,NaN,2020.0,NaN,1623.0,4100.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11549055,N,Y,25.921979,-80.121703,6,blue,15701 COLLINS CONDO at 15701 Collins Ave #2601...
6,PAST SALE,May-10-2024,Condo/Co-op,2901 Collins Ave # 1205,Miami Beach,FL,33140,4995000.0,2.0,2.5,The Edition,1450.0,None,2014.0,None,3445.0,NaN,Sold,NA,NA,https://www.redfin.com/FL/Miami-Beach/2901-Col...,Manual Entry,A11456329,NA,NA,25.805127,-80.124117,7,blue,The Edition at 2901 Collins Ave # 1205 in Miam...
7,PAST SALE,May-20-2024,Condo/Co-op,16901 Collins Ave #4305,Sunny Isles Beach,FL,33160.0,4500000.0,3.0,4.5,JADE SIGNATURE CONDO,2639.0,NaN,2018.0,NaN,1705.0,4616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11431905,N,Y,25.932756,-80.121005,8,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #430...
8,PAST SALE,May-15-2024,Condo/Co-op,17875 Collins Ave #3706,Sunny Isles Beach,FL,33160.0,3800000.0,4.0,4.0,ACQUALINA OCEAN RESIDENCE,2736.0,NaN,2006.0,NaN,1389.0,5190.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11527429,N,Y,25.941240,-80.120744,9,blue,ACQUALINA OCEAN RESIDENCE at 17875 Collins Ave...
9,PAST SALE,May-15-2024,Condo/Co-op,10295 Collins Ave #403,Bal Harbour,FL,33154.0,3700000.0,3.0,3.5,10295 COLLINS AVE RESDNTA,3105.0,NaN,2007.0,NaN,1192.0,5101.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,MARMLS,A11530469,N,Y,25.898952,-80.123405,10,blue,10295 COLLINS AVE RESDNTA at 10295 Collins Ave...


## Time on Market Calculator

In [43]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 8, 22) ## List (Earlier) date
date2 = datetime(2024, 5, 1) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

253
